# Assignment 3
Student Name: Ziyi Jiang  |  Student ID: 634926886  |  UPI: zjia631

## Section 1 -- Report

###  1. Data Representation

#### Method
The data representation I used in this assignment is Bag of Words. <br/>
Firstly, I looped through the abstracts and combined them based on class, so I got all the text for each class. Then, for each class, I split the corresponding text and count the number of each kind of word. In the end, I got a nested dictionary. The outer dictionary has the class as the key and corresponding words' occurrence number as the value. The inner dictionary has the word as the key and the corresponding occurrence number in the class as the value. <br/>
Here is I record the BoW representation: {class{word: number of the word in the class, ...}, ...})


#### Reason
As a vast amount of data and I do not have much knowledge about it, it would be difficult to use other representations such as the top frequent words, N-grams. Also, the amount of each class is not equally split. Class B and E take over 90% of the dataset. Hence, using top frequent words may cause bias on the classifier. Therefore, I decided to use the bag of words data representation.


### 2. Data Preprocessing

#### Preprocessing
1. np.genfromtxt is used to read the CSV file, and then the data is split based on different column.
2. All the data is stored as string in NumPy ndarrys.
3. The individual id and class are stored as one string. For example: "1".
4. The ids and abstractes of tst.csv are stored. For trg.csv, only the classes and abstractes are stored.
5. The abstracte is splited into substrings by " ".
6. The classifier then uses those data to find all needed parameters. (detail shown in Implementation section) <br/>
    \- a list of unique class &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \- total number of unique words &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \- total number of classes <br/>
    \- number of a word in a class &nbsp;&nbsp;&nbsp; \- total number of each class &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \- total number of words in a class <br/>

#### Reason
The id and class are a single character, so there is no need to split them. For tst.csv we need the ids to produce the corresponding prediction. However, we do not need ids for trg.csv. We only need to check are the predictions are correct for trg.csv. Hence, the ids of trg.csv are not needed. We need to count the different words in the abstract to calculate the Naive Bayes result, so the abstract needs to be split.

### 3. Extension
The Extension used in this report is Complement Naive Bayes.

#### Reason
Here is the data of the trg.csv: <br/>
\- Number of each class: {'A': 128, 'B': 1602, 'E': 2144, 'V': 126} <br/>
\- Total word number of each class: {'A': 27529, 'B': 285505, 'E': 379088, 'V': 22700} <br/>
We can see that both the class number and word number of B and E are significantly more than A and V. This will cause the skewed data bias, which makes the Standard Naive Bayes classifier predict more to B and E as the P(A) and P(V) would be much smaller than P(B) and P(E). <br/>
However, the Complement Naive Bayes classifier uses the probability of the class divides the probability of a word that occurred in other classes -- P(class) / P(abstracte|other class), which could lower the influence of the skewed data bias and increase the accuracy.

For example, if P(abstracte|class1) = 0.8 and P(abstracte|class2) = 0.2 but P(class1) = 0.1 and P(class2)= 0.6. And P(abstracte|classes other than class1) = 0.1 and P(abstracte|classes other than class2) = 0.9 Then, because P(class1) * P(abstracte|class1) = 0.08 < P(class2) * P(abstracte|class2) = 0.12, the Standard Naive Bayes classifier would predict this abstracte as class2. However, the Complement Naive Bayes classifier would calculates P(class1) / P(abstracte|classes other than class1) = 1 > P(class2) / P(abstracte|classes other than class2) = 0.667, and hence assign this abstracte to class1.

Therefore, since there is significantly skewed data bias in this dataset, Complement Naive Bayes is a suitable extension.

### 4. Implementation
Due to the 0 probability problem, I use 1 and total unique words number as smoothing parameters:<br/>
&nbsp;&nbsp;&nbsp;&nbsp;
P(class|abstracte) = P(class) $\times$ P(abstracte|class) = P(class) $\times$ $\prod_{}^{}$P(word|class) = P(class) $\times$ $\prod_{}^{}$$\frac{\text{word number } + 1}{\text{total words number in class } + \text{ total unique words number}}$

Due to the underflow problem, the code uses np.log() to record the probabilities (log-recorded probability):<br/>
&nbsp;&nbsp;&nbsp;&nbsp;
log-recorded P(class|abstracte) = log-recorded P(class) + log-recorded P(abstracte|class) <br/>
&nbsp;&nbsp;&nbsp;&nbsp;
= log-recorded P(class) + $\sum_{}^{}$log-recorded P(word|class) <br/> 
&nbsp;&nbsp;&nbsp;&nbsp;
= $\log$$\frac{\text{number of the class}}{\text{total number of classes}}$ + $\sum_{}^{}$$\log$$\frac{\text{word number } + 1}{\text{total words number in class } + \text{ total unique words number}}$

For both classifier, I created two classes called StandardNaiveBayesClassifier and ComplementNaiveBayesClassifier. There are two methods in each class -- fit() and predict(). The fit() method record 6 parameters that are needed by the Naive Bayes. The predict method uses those parameters to implement different vision of Naive Bayes.

#### Standard Naive Bayes Classifier
The Standard Naive Bayes classifier uses the formula: P(class|abstracte) = P(class) * P(abstracte|class) <br/>
***Detail:*** <br/>
Firstly, I recorded 6 parameter:
1. a list of unique class (["A", "B", "E", "V"]) &nbsp;&nbsp;&nbsp; 4. total number of unique words (a dictionary -- {word: number of the word, ...})
2. total number of classes (an integer) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 5. number of a word in a class (a dictionary of dictionary -- {class{word: number of the word in the class, ...}, ...})
3. total number of each class (an integer) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 6. total number of words in a class (an integer)

Then. I use those parameters to calculate the log-recorded P(class|abstracte) of each class:
1. calculate log-recorded P(class) by using np.log(number of the class / total number of classes)
2. calculate log-recorded P(abstracte|class) by sum all log(P(word|class)) of each word in the abstract
3. use log(P(class)) + log(P(abstracte|class)) to get the log-recorded P(class|abstracte)
4. find the class with the maximum log-recorded P(class|abstracte) -- this would be the predicted class

#### Complement Naive Bayes Classifier
The Complement Naive Bayes classifier uses the formula: P(class|abstracte) = P(class) / P(abstracte|other class) <br/>
***Detail:*** <br/>
Firstly, I recorded the same 6 parameters as in the Standard Naive Bayes classifier.<br/>
Then, I calculate the P(class|abstracte) of each class:
1. the calculation of log-recorded P(class) is the same as in the Standard Naive Bayes classifier
2. calculate log-recorded P(abstracte|other class) by sum all log(P(word|other class)) of each word in the abstract:
    - calculate P(word|other class) by using <br/> (sum the number of the word in all other classes + 1) / (sum the total number of words in all other classes + total number of unique words)
3. use log(P(class)) - log(P(abstracte|other class)) to get the log-recorded P(class|abstracte)
4. find the class with the maximum log-recorded P(class|abstracte) -- this would be the predicted class

### 5. Result

#### Test Method (Training and Validation)
Split Method: <br/>
I wrote a function named stratified_cv_split() which split the data into k folds based on the percentage of each class. The function first split the indices based on different classes. Then, it randomly split those indices into k subgroups for each class. After that, the function uses those indices to extract the corresponding abstract and class. <br/>
Because the function uses stratified cross-validation, the split data will have similar distribution with the whole data.

Get Accuracy Method: <br/>
The get_accuracy() function converts the result into true and false based on the condition (actual class = predicted class). Then, the function counts how many of the results are True using a sum() function. And use the count / number of results to get and return the accuracy.

#### Comparision

I used a loop to do the stratified cross-validation 20 times. In each loop, I use stratified_cv_split() to do 5-folds stratified cross-validation, use the same split data for the two classifiers, and record the accuracies. After the loop, I calculate the mean and make the comparison. Here is the test result on the training set: <br/>
Standard Naive Bayes Classifier mean accuracy: 0.9439314871336758 <br/>
Complement Naive Bayes mean accuracy: 0.9617133179483361 <br/>
Difference of mean accuracies (Complement - Standard): 0.017781830814660338 <br/>

For the accuracy of tst.csv, Kaggle shows the Standard Naive Bayes classifier's accuracy is 0.95333 and Complement Naive Bayes classifier's accuracy is 97.

Therefore, for this dataset, the Complement Naive Bayes classifier is more accurate than the Standard Naive Bayes classifier.

## Section 2 -- Code

In [1]:
import numpy as np

### Standard Naive Bayes Classifier

In [2]:
class NaiveBayesClassifier:

    def __init__(self):
        """
        Unique class is stored as self.unique_class.
        Total number of classes is stored as self.total_classes_number.
        Total number of each class is stored in self.class_number_dict.
        Total number of unique words is stored as self.unique_word_number.
        Total number of a word in a class is stored in self.class_word_number_dict.
        otal number of a word in a class is stored in self.class_total_words_dict.
        """
        self.unique_classes = []
        self.total_classes_number = 0
        self.class_number_dict = {}
        self.unique_word_number = 0
        self.class_word_number_dict = {}
        self.class_total_words_dict = {}

    def fit(self, abstracts, classes):
        classes = np.array(classes)
        abstracts = np.array(abstracts)
        # unique class
        self.unique_classes = sorted(set(classes))

        # total number of classes
        self.total_classes_number = len(classes)

        # total number of each class
        c, counts = np.unique(classes, return_counts=True)
        self.class_number_dict = dict(zip(c, counts))

        # total number of unique words
        all_text = ""
        for text in abstracts:
            all_text = all_text + text + " "
        all_text = all_text.split()
        unique_word = set(all_text)
        self.unique_word_number = len(unique_word)

        # total number of a word in a class
        self.class_word_number_dict = {}
        class_text_dict = {}
        for c in self.unique_classes:
            class_text_dict[c] = ""
        for i in range(len(abstracts)):
            c = classes[i]
            text = abstracts[i]
            class_text_dict[c] = class_text_dict[c] + text + " "
        for c in class_text_dict:
            text = np.array(class_text_dict[c].split())
            word, counts = np.unique(text, return_counts=True)
            self.class_word_number_dict[c] = dict(zip(word, counts))

        # total number of words in a class
        self.class_total_words_dict = {}
        for c in class_text_dict:
            text = class_text_dict[c].split()
            self.class_total_words_dict[c] = len(text)

    def predict(self, abstracts, ids):
        result = []
        for i in range(len(ids)):
            pred_id = ids[i]
            pred_abs = abstracts[i].split()
            id_result_prob = []
            for c in self.unique_classes:
                prob_c = np.log(self.class_number_dict[c] / self.total_classes_number)
                prob_x = 0
                word_number_dict = self.class_word_number_dict[c]
                total_words_number = self.class_total_words_dict[c]
                for word in pred_abs:
                    if word in word_number_dict:
                        word_number = word_number_dict[word]
                    else:
                        word_number = 0
                    prob_word = np.log((word_number + 1) / (total_words_number + self.unique_word_number))
                    prob_x = prob_x + prob_word
                prob = prob_c + prob_x
                id_result_prob.append([c, prob])

            id_result_prob = sorted(id_result_prob, key=lambda x: x[1])
            result.append((pred_id, id_result_prob[-1][0]))

        return result

### Navie Bayes Classifier with Extension

#### Complement Naive Bayes

In [3]:
class ComplementNaiveBayesClassifier:

    def __init__(self):
        self.unique_classes = []
        self.total_classes_number = 0
        self.class_number_dict = {}
        self.unique_word_number = 0
        self.class_word_number_dict = {}
        self.class_total_words_dict = {}

    def fit(self, abstracts, classes):
        classes = np.array(classes)
        abstracts = np.array(abstracts)
        
        self.unique_classes = sorted(set(classes))

        self.total_classes_number = len(classes)

        c, counts = np.unique(classes, return_counts=True)
        self.class_number_dict = dict(zip(c, counts))

        all_text = ""
        for text in abstracts:
            all_text = all_text + text + " "
        all_text = all_text.split()
        unique_word = set(all_text)
        self.unique_word_number = len(unique_word)

        self.class_word_number_dict = {}
        class_text_dict = {}
        for c in self.unique_classes:
            class_text_dict[c] = ""
        for i in range(len(abstracts)):
            c = classes[i]
            text = abstracts[i]
            class_text_dict[c] = class_text_dict[c] + text + " "
        for c in class_text_dict:
            text = np.array(class_text_dict[c].split())
            word, counts = np.unique(text, return_counts=True)
            self.class_word_number_dict[c] = dict(zip(word, counts))

        self.class_total_words_dict = {}
        for c in class_text_dict:
            text = class_text_dict[c].split()
            self.class_total_words_dict[c] = len(text)

    def predict(self, abstracts, ids):
        result = []
        for i in range(len(ids)):
            pred_id = ids[i]
            pred_abs = abstracts[i].split()
            id_result_prob = []
            for c in self.unique_classes:
                prob_c = np.log(self.class_number_dict[c] / self.total_classes_number)
                prob_x = 0
                """
                Extension
                """
                not_cs = []
                word_number_in_ncs = []
                total_words_number = 0
                for nc in self.unique_classes:
                    if nc != c:
                        not_cs.append(nc)
                        word_number_in_ncs.append(self.class_word_number_dict[nc])
                        total_words_number += self.class_total_words_dict[nc]  
                for word in pred_abs:
                    word_number = 0
                    for word_number_dict in word_number_in_ncs:
                        if word in word_number_dict:
                            word_number += word_number_dict[word]
                    prob_word = np.log((word_number + 1) / (total_words_number + self.unique_word_number))
                    prob_x = prob_x + prob_word
                prob = prob_c - prob_x
                id_result_prob.append([c, prob])
            id_result_prob = sorted(id_result_prob, key=lambda x: x[1])
            result.append((pred_id, id_result_prob[-1][0]))

        return result

### Test

In [4]:
def stratified_cv_split(words, classes, randomseed, k=5):
    words = np.array(words)
    classes = np.array(classes)
    class_index_dict = {}
    for i in range(len(classes)):
        if classes[i] in class_index_dict:
            class_index_dict[classes[i]].append(i)
        else:
            class_index_dict[classes[i]] = [i]
    class_split_result = []
    word_split_result = []
    split_percent = 10 / k / 10
    for c in class_index_dict:
        randomseed += 1
        c_indices = np.array(class_index_dict[c])
        rng = np.random.default_rng(seed = randomseed)
        indices = rng.permutation(len(c_indices))
        c_split_result = []
        w_split_result = []
        split_point1 = 0
        split_point2 = int(len(c_indices) * split_percent)
        for i in range(k):
            if i == k-1:
                indexs = indices[split_point1:]
                indexs = c_indices[indexs]
                c_split_result.append(classes[indexs])
                w_split_result.append(words[indexs])
            else:
                indexs = indices[split_point1:split_point2]
                indexs = c_indices[indexs]
                c_split_result.append(classes[indexs])
                w_split_result.append(words[indexs])
            split_part = int(len(c_indices) * split_percent)
            split_point1 += split_part
            split_point2 += split_part
        class_split_result.append(c_split_result)
        word_split_result.append(w_split_result)
    result = []
    for i in range(k):
        x_test = np.array([])
        y_test = np.array([])
        x_train = np.array([])
        y_train = np.array([])
        for c in range(len(class_split_result)):
            x_test = np.concatenate((x_test, word_split_result[c][i]))
            y_test = np.concatenate((y_test, class_split_result[c][i]))
        for j in range(k):
            if j != i:
                for c in range(len(class_split_result)):
                    x_train = np.concatenate((x_train, word_split_result[c][j]))
                    y_train = np.concatenate((y_train, class_split_result[c][j]))
        result.append((x_train, x_test, y_train, y_test))
    return result

In [5]:
def get_accuracy(result):
    result = np.array(result)
    count = sum(result[:, 0] == result[:, 1])
    accuracy = count / len(result)
    return accuracy

#### Read Data

In [6]:
classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

#### Comparision

In [7]:
nbc_scores = []
cnbc_scores = []
for i in range(20):
    scv_split = stratified_cv_split(train_abstracts, classes, i)
    for data in scv_split:
        x_train = data[0]
        x_test = data[1]
        y_train = data[2]
        y_test = data[3]
        nbc =NaiveBayesClassifier()
        nbc.fit(x_train, y_train)
        nbc_result = nbc.predict(x_test, y_test)
        nbc_accuracy = get_accuracy(nbc_result)
        nbc_scores.append(nbc_accuracy)
    
        cnbc =ComplementNaiveBayesClassifier()
        cnbc.fit(x_train, y_train)
        cnbc_result = cnbc.predict(x_test, y_test)
        cnbc_accuracy = get_accuracy(cnbc_result)
        cnbc_scores.append(cnbc_accuracy)

print("Standard Naive Bayes Classifier mean:", np.mean(nbc_scores))
print("Complement Naive Bayes mean:", np.mean(cnbc_scores))
print("Difference of Means (Complement - Standard):", np.mean(cnbc_scores) - np.mean(nbc_scores))

Standard Naive Bayes Classifier mean: 0.9439314871336758
Complement Naive Bayes mean: 0.9617133179483361
Difference of Means (Complement - Standard): 0.017781830814660338


### Final Result of tst.csv

#### Standard Naive Bayes

In [8]:
train_classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
train_classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

ids = []
abstracts = []
data = np.genfromtxt("tst.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(0, 1))
ids = data[:, 0]
abstracts = data[:, 1]

nbc = NaiveBayesClassifier()
nbc.fit(train_abstracts, train_classes)
result = nbc.predict(abstracts, ids)

f = open("nbc_result.csv", "a")
f.write("id,class\n")
for r in result:
    line = r[0] + "," + r[1] + "\n"
    f.write(line)
f.close()

#### Complement Naive Bayes

In [9]:
train_classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
train_classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

ids = []
abstracts = []
data = np.genfromtxt("tst.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(0, 1))
ids = data[:, 0]
abstracts = data[:, 1]

cnbc = ComplementNaiveBayesClassifier()
cnbc.fit(train_abstracts, train_classes)
result = cnbc.predict(abstracts, ids)
f = open("cnbc_result.csv", "a")
f.write("id,class\n")
for r in result:
    line = r[0] + "," + r[1] + "\n"
    f.write(line)
f.close()